In [1]:
import pandas as pd
import os

# Base path
base_read_path = "../../results/pipeline results/5884paras_598qna/copies/"
lqr_path = os.path.join(base_read_path, "LQR_processed_results - Copy.csv")

# Read data
lqr_df = pd.read_csv(lqr_path)

# Map level to true label
def get_true_label(level):
    if level.strip().lower() == "easy":
        return "singlehop"
    elif level.strip().lower() in ["medium", "hard"]:
        return "multihop"
    return None  # for safety

lqr_df["true_label"] = lqr_df["level"].apply(get_true_label)

# Strip/clean predicted classification as well
lqr_df["classification"] = lqr_df["classification"].str.strip().str.lower()

# Compute correctness
lqr_df["is_correct"] = lqr_df["classification"] == lqr_df["true_label"]

# Overall accuracy
overall_accuracy = lqr_df["is_correct"].mean()

# Accuracy by level
level_grouped = lqr_df.groupby("level")["is_correct"].agg(['count', 'sum'])
level_grouped["accuracy"] = level_grouped["sum"] / level_grouped["count"]

# Display results
print("Overall Accuracy: {:.2f}%".format(overall_accuracy * 100))
print("\nAccuracy by Level:")
print(level_grouped[["accuracy"]].rename(columns={"accuracy": "Accuracy (%)"}) * 100)


Overall Accuracy: 26.09%

Accuracy by Level:
        Accuracy (%)
level               
easy       83.673469
hard       16.400000
medium     13.200000
